# The Spinning Effective One-Body Hamiltonian

### Author: Tyler Knowles

## This module documents the reduced spinning effective one-body Hamiltonian as numerically implemented in LALSuite's SEOBNRv3 gravitational waveform approximant.

### Throughout this module, we will refer to [Barausse and Buonanno (2010)](https://arxiv.org/abs/0912.3517) as BB2010 and [Barausse and Buonanno (2011)](https://arxiv.org/abs/1107.2904) as BB2011.

<font color='green'>**This module has been validated against the LALSuite SEOBNRv3/SEOBNRv3_opt code that was reviewed and approved for LIGO parameter estimation by the LIGO Scientific Collaboration.**</font>

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows:

0. [Introduction](#intro): The Physical System of Interest
1. [Step 1](#Hreal): The Real Hamiltonian $H_{\rm real}$
1. [Step 2](#Heff): The Effective Hamiltonian $H_{\rm eff}$
1. [Step 3](#Heff_terms): Terms of $H_{\rm eff}$  
    a. [Step 3a](#Hs): Leading Order Spin Effects $H_{\rm S}$  
    b. [Step 3b](#Hns): The Nonspinning Hamiltonian $H_{\rm NS}$  
    c. [Step 3c](#Hd): The Quadrupole Deformation $H_{\rm D}$
1. [Step 4](#Hso): The Spin-Orbit Term $H_{\rm SO}$  
    a. [Step 4a](#HsoTerm1): $H_{\rm SO}$ Term 1  
    b. [Step 4b](#HsoTerm2coeff): $H_{\rm SO}$ Term 2 Coefficient  
    c. [Step 4c](#HsoTerm2): $H_{\rm SO}$ Term 2
1. [Step 5](#Hss): The Spin-Spin Term $H_{\rm SS}$
1. [Step 6](#HsoHsscommon): Terms Common to $H_{\rm SO}$ and $H_{\rm SS}$
1. [Step 7](#HnsTerms): The $H_{\rm NS}$ Terms
1. [Step 8](#HdTerms): The $H_{\rm D}$ Terms  
    a.[Step 8a](#Hdcoeff): $H_{\rm D}$ Coefficient  
    b.[Step 8b](#Hdsum): $H_{\rm D}$ Sum
1. [Step N-5](#momentum_terms): The Momentum Terms
1. [Step N-4](#alphabetagamma): The Metric-Dependent Terms $\alpha$, $\beta^{i}$, and $\gamma^{ij}$
1. [Step N-3](#Hreal_spin_combos): $H_{\rm real}$ Spin Combinations ${\bf S}^{*}$ and ${\bf S}_{\rm Kerr}$
1. [Step N-2](#Heff_spin_combos): $H_{\rm eff}$ Spin Combinations $\boldsymbol{\sigma}$ and $\boldsymbol{\sigma}^{*}$
1. [Step N-1](#mass_terms): Mass Terms $\mu$, $M$, and $\eta$
1. [Step N](#metric): The Metric $g$ of Choice
1. [Notes](#notes)
1. [Output](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF

<a id='intro'></a>

# Introduction: The Physical System of Interest \[Back to [top](#toc)\]
$$\label{intro}$$

Consider two black holes with masses $m_{1}$, $m_{2}$ and spins ${\bf S}_{1}$, ${\bf S}_{2}$ in a binary system.  The spinning effective one-body ("SEOB") Hamiltonian [$H_{\rm real}$](#Hreal) describes the dynamics of this system; we will define [$H_{\rm real}$](#Hreal) as in BB2010 Section VE.  There, [$H_{\rm real}$](#Hreal) is canonically transformed and mapped to an effective Hamiltonian [$H_{\rm eff}$](#Heff) describing the motion of a test particle of mass [$\mu$](#mass_terms) and spin [${\bf S}_{\rm Kerr}$](#Hreal_spin_combos).  Here we seek to break up [$H_{\rm real}$](#Hreal) and document the terms in such a way that the resulting Python code can be used to numerically evaluate [$H_{\rm real}$](#Hreal).  

We will use Greek letters to denote spacetime tensor indicies, and spacial tensor indicies with lowercase Latin letters.

<font color='red'>Please note: LALSuite computes $H_{\rm real}$ up to a constant multiple of $M$ (*I think*).</font>

<a id='Hreal'></a>

# Step 1: The real Hamiltonian $H_{\textrm{real}}$ \[Back to [top](#toc)\]
$$\label{Hreal}$$

The SEOB Hamiltonian $H_{\rm real}$ is given by Equation (5.69) of BB2010:

\begin{equation}
    H_{\rm real} = M \sqrt{ 1 + 2 \eta \left( \frac{ H_{\rm eff} }{ \mu } - 1 \right) }.
\end{equation}

Here [$H_{\rm eff}$](#Heff) is an [*effective* Hamiltonian](#intro) and [$M$](#mass_terms), [$\mu$](#mass_terms), and [$\eta$](#mass_terms) are constants determined by $m_{1}$ and $m_{2}$.

<font color='red'>The code in LALSimIMRSpinEOBHamiltonianPrec.c claims the origin of $H_{\textrm{real}}$ is Equation (2) of [Taracchini, Pan, Buonanno, et al](https://arxiv.org/abs/1202.0790):

\begin{equation}
    H_{\rm real} = M \sqrt{ 1 + 2 \nu \left( \frac{ H_{\rm eff} }{ \mu } - 1 \right) } - M
\end{equation}

where $\nu = \eta = \frac{\mu}{M}$.  Double-check this!</font>

In [1]:
%%writefile SEOBNR/Hamiltonian.txt
Hreal = M * sp.sqrt(1 + 2*eta*(Heff/mu - 1))

Overwriting SEOBNR/Hamiltonian.txt


<a id='Heff'></a>

# Step 2: The Effective Hamiltonian $H_{\rm eff}$ \[Back to [top](#toc)\]
$$\label{Heff}$$

The effective Hamiltonian $H_{\rm eff}$ is given by Equation (5.70) of BB2010.  We combine the notation of BB2010 Equations (5.70) and (5.43) with our notation [$H_{\rm D}$](#Hd) to write

\begin{equation}
    H_{\rm eff} = H_{\rm S} + H_{\rm NS} \underbrace{ -\frac{ \eta }{ 2 r^{3} } \left( \delta^{ij} - 3 n^{i} n^{j} \right) S^{*}_{i} S^{*}_{j} }_{H_{\rm D}}.
\end{equation}

Here [$H_{\rm S}$](#Hs) denotes leading order effects of spin-spin and spin-orbit coupling, [$H_{\rm NS}$](#Hns) is the Hamiltonian for a nonspinning test particle, and [$H_{\rm D}$](#Hd) describes quadrupole deformation of the coupling of the particle's spin with itself to leading order.

In [2]:
%%writefile -a SEOBNR/Hamiltonian.txt

Heff = Hs + Hns + Hd

Appending to SEOBNR/Hamiltonian.txt


<a id='Heff_terms'></a>

# Step 3: Terms of $H_{\rm eff}$ \[Back to [top](#toc)\]
$$\label{Heff_terms}$$

In this step, we break down each of the terms [$H_{\rm S}$](#Hs), [$H_{\rm NS}$](#Hns), and [$H_{\rm D}$](#Hd) in [$H_{\rm eff}$](#Heff).

<a id='Hs'></a>

## Step 3a: Leading Order Spin Effects $H_{\rm S}$ \[Back to [top](#toc)\]
$$\label{Hs}$$

From BB2010 Equation (4.17),

\begin{equation}
    H_{\rm S} = H_{\rm SO} + H_{\rm SS}
\end{equation}

where [$H_{\rm SO}$](#Hso) includes spin-orbit terms and [$H_{\rm SS}$](#Hss) includes spin-spin terms.

In [3]:
%%writefile -a SEOBNR/Hamiltonian.txt

Hs = Hso + Hss

Appending to SEOBNR/Hamiltonian.txt


<a id='Hns'></a>

## Step 3b: The Nonspinning Hamiltonian $H_{\rm NS}$ \[Back to [top](#toc)\]
$$\label{Hns}$$

From BB2010 Equation (5.43),

\begin{equation}
    H_{\rm NS} = \underbrace{ \beta^{i} p_{i} }_{\beta p\ {\rm sum}} + \alpha \sqrt{ \smash[b]{\underbrace{ \mu^{2} + \gamma^{ij} p_{i} p_{j} + {\cal Q}_{4} \left( p \right) }_{H_{\rm NS}\ {\rm radicand}}}}
\end{equation}

For further magic, see [$\beta p\ {\rm sum}$](#betapsum), [$\alpha$](#alpha), and [$H_{\rm NS}\ {\rm radicand}$](#Hnsradicand). 

In [4]:
%%writefile -a SEOBNR/Hamiltonian.txt

Hns = betapsum + alpha*sp.sqrt(Hnsradicand)

Appending to SEOBNR/Hamiltonian.txt


<a id='Hd'></a>

## Step 3c: The Quadrupole Deformation $H_{\rm D}$ \[Back to [top](#toc)\]
$$\label{Hd}$$

We break up $H_{\rm D}$ as follows:

\begin{equation}
    H_{\rm D} = \underbrace{ -\frac{ \eta }{ 2 r^{3} } }_{H_{\rm D}\ {\rm coefficient}} \underbrace{ \left( \delta^{ij} - 3 n^{i} n^{j} \right) S^{*}_{i} S^{*}_{j} }_{H_{\rm D}\ {\rm sum}}
\end{equation}

For further dissection, see [$H_{\rm D}\ {\rm coefficient}$](#Hdcoeff) and [$H_{\rm D}\ {\rm sum}$](#Hdsum).

In [5]:
%%writefile -a SEOBNR/Hamiltonian.txt

Hd = Hdcoeff*Hdsum

Appending to SEOBNR/Hamiltonian.txt


<a id='Hso'></a>

# Step 4: The Spin-Orbit Term $H_{\rm SO}$ \[Back to [top](#toc)\]
$$\label{Hso}$$

From BB2010 Equation (4.18):

\begin{align}
    H_{\rm SO} &= \underbrace{ \frac{ e^{2 \nu - \tilde{\mu} } \left( e^{\tilde{\mu} + \nu} - \tilde{B} \right) \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right) \left( {\bf S} \cdot \hat{\bf S}_{\rm Kerr} \right) }{ \tilde{B}^{2} \sqrt{Q} \xi^{2} } }_{\rm Term\ 1} + \underbrace{ \frac{ e^{\nu - 2 \tilde{\mu}} }{ \tilde{B}^{2} \left( \sqrt{Q} + 1 \right) \sqrt{Q} \xi^{2} } }_{\rm Term\ 2\ coefficient} \left\{ \left( {\bf S} \cdot \boldsymbol{\xi} \right) \tilde{J} \left[ \mu_r \left( \hat{\bf p} \cdot {\bf v} r \right) \left( \sqrt{Q} + 1 \right) - \mu_{\cos \theta} \left( \hat{\bf p} \cdot {\bf n} \right) \xi^{2} \right. \right. \\
    & \left. -\sqrt{Q} \left( \nu_r \left( \hat{\bf p} \cdot {\bf v} r \right) + \left( \mu_{\cos \theta} - \nu_{\cos \theta} \right) \left( \hat{\bf p} \cdot {\bf n} \right) \xi^{2} \right) \right] \tilde{B}^{2} + e^{\tilde{\mu} + \nu} \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right) \left( 2 \sqrt{Q} + 1 \right) \left[ \tilde{J} \nu_r \left( {\bf S} \cdot {\bf v} \right) - \nu_{\cos \theta} \left( {\bf S} \cdot {\bf n} \right) \xi^{2} \right] \tilde{B} \\
    & \left. -\tilde{J} \tilde{B}_{r} e^{\tilde{\mu} + \nu} \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right) \left( \sqrt{Q} + 1 \right) \left( {\bf S} \cdot {\bf v} \right) \right\}.
\end{align}

We will denote by Term 2 the expression in $\left\{ \right\}$.  For further information, see [$H_{\rm SO}\ {\rm Term\ 1}$](#HsoTerm1), [$H_{\rm SO}\ {\rm Term\ 2\ coefficient}$](#HsoTerm2coeff), and [$H_{\rm SO}\ {\rm Term\ 2}$](#HsoTerm2).

In [6]:
%%writefile -a SEOBNR/Hamiltonian.txt

Hso = HsoTerm1 + HsoTerm2coeff*HsoTerm2

Appending to SEOBNR/Hamiltonian.txt


<a id='HsoTerm1'></a>
## Step 4a: $H_{\rm SO}$ Term 1 \[Back to [top](#toc)\]
$$\label{HsoTerm1}$$

We have

\begin{equation}
    H_{\rm SO}\ {\rm Term\ 1} = \frac{ e^{2 \nu - \tilde{\mu} } \left( e^{\tilde{\mu} + \nu} - \tilde{B} \right) \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right) \left( {\bf S} \cdot \hat{\bf S}_{\rm Kerr} \right) }{ \tilde{B}^{2} \sqrt{Q} \xi^{2} }
\end{equation}

We divide $H_{\rm SO}\ {\rm Term\ 1}$ into [numerator](#HsoTerm1num) and [denominator](#HsoTerm1den).

In [7]:
%%writefile -a SEOBNR/Hamiltonian.txt

HsoTerm1 = HsoTerm1num/HsoTerm1den

Appending to SEOBNR/Hamiltonian.txt


<a id='HsoTerm1num'></a>

### $H_{\rm SO}$ Term 1 Numerator \[Back to [top](#toc)\]
$$\label{HsoTerm1num}$$

The numerator of $H_{\rm SO}$ Term 1 is

\begin{equation}
    H_{\rm SO}\ {\rm Term\ 1\ numerator} = \underbrace{e^{2 \nu - \tilde{\mu}}}_{exp2nuexpmu} \left( \smash[b]{ \underbrace{ e^{\tilde{\mu} + \nu} - \tilde{B} }_{expmuexpnuminusB} } \right) \left( \smash[b]{ \underbrace{ \hat{\bf p} \cdot \boldsymbol{\xi} r }_{pdotxir} } \right) \left( \smash[b]{ \underbrace{ {\bf S} \cdot \hat{\bf S}_{\rm Kerr} }_{SdotSkerr} } \right)
\end{equation}

We break $H_{\rm SO}$ Term 1 Numerator into four terms: [$e^{2 \nu - \tilde{\mu} }$](#FIXME),  [$e^{\tilde{\mu} + \nu} - \tilde{B}$](#FIXME), [$\hat{\bf p} \cdot \boldsymbol{\xi} r$](#FIXME), and [${\bf S} \cdot \hat{\bf S}_{\rm Kerr}$](#FIXME).

In [8]:
%%writefile -a SEOBNR/Hamiltonian.txt

HsoTerm1num = exp2nuexpmu*expmuexpnuminusB*pdotxir*SdotSkerr

Appending to SEOBNR/Hamiltonian.txt


<a id='HsoTerm1den'></a>

### $H_{\rm SO}$ Term 1 Denominator \[Back to [top](#toc)\]
$$\label{HsoTerm1den}$$

The denominator of $H_{\rm SO}$ Term 1 is

\begin{equation}
    H_{\rm SO}\ {\rm Term\ 1\ denominator} = \tilde{B}^{2} \sqrt{Q} \xi^{2}
\end{equation}

We define [$\tilde{B}$](#Btilde), [$Q$](#Q), and [$\xi$](#xi).

In [9]:
%%writefile -a SEOBNR/Hamiltonian.txt

HsoTerm1den = Btilde*Btilde*sp.sqrt(Q)*xi*xi

Appending to SEOBNR/Hamiltonian.txt


<a id='HsoTerm2coeff'></a>
## Step 4b: $H_{\rm SO}$ Term 2 Coefficient \[Back to [top](#toc)\]
$$\label{HsoTerm2coeff}$$

We have

\begin{equation}
    H_{\rm SO}\ {\rm Term\ 2\ coefficient} = \frac{ e^{\nu - 2 \tilde{\mu}} }{ \tilde{B}^{2} \left( \sqrt{Q} + 1 \right) \sqrt{Q} \xi^{2} }
\end{equation}

We divide $H_{\rm SO}$ Term 2 coefficient into [numerator](#HsoTerm2coeffnum) and [denominator](#HsoTerm2coeffden).

In [10]:
%%writefile -a SEOBNR/Hamiltonian.txt

HsoTerm2coeff = HsoTerm2coeffnum/HsoTerm2coeffden

Appending to SEOBNR/Hamiltonian.txt


<a id='HsoTerm2coeffnum'></a>

### $H_{\rm SO}$ Term 2 Coefficient Numerator \[Back to [top](#toc)\]
$$\label{HsoTerm2coeffnum}$$

The numerator of $H_{\rm SO}$ Term 2 coefficient is

\begin{equation}
    H_{\rm SO}\ {\rm Term\ 2\ coefficient\ numerator} = e^{\nu - 2 \tilde{\mu}}.
\end{equation}

We define [$e^{\nu - 2 \tilde{\mu}}$](#expnuexp2mu).

In [11]:
%%writefile -a SEOBNR/Hamiltonian.txt

HsoTerm2coeffnum = expnuexp2mu

Appending to SEOBNR/Hamiltonian.txt


<a id='HsoTerm2coeffden'></a>

### $H_{\rm SO}$ Term 2 Coefficient Denominator \[Back to [top](#toc)\]
$$\label{HsoTerm2coeffden}$$

The denominator of $H_{\rm SO}$ Term 2 coefficient is

\begin{equation}
    H_{\rm SO}\ {\rm Term\ 2\ coefficient\ denominator} = \tilde{B}^{2} \left( \sqrt{Q} + 1 \right) \sqrt{Q} \xi^{2}.
\end{equation}

We define [$\tilde{B}$](#Btilde), [$Q$](#Q), and [$\xi$](#xi).

In [12]:
%%writefile -a SEOBNR/Hamiltonian.txt

HsoTerm2coeffden = Btilde*Btilde*(Q+1)*sp.sqrt(Q)*xi*xi

Appending to SEOBNR/Hamiltonian.txt


<a id='HsoTerm2'></a>
## Step 4c: $H_{\rm SO}$ Term 2 \[Back to [top](#toc)\]
$$\label{HsoTerm2}$$

We have

\begin{equation}
    H_{\rm SO}\ {\rm Term\ 2} = \underbrace{ \left( {\bf S} \cdot \boldsymbol{\xi} \right) \tilde{J} \left[ \mu_r \left( \hat{\bf p} \cdot {\bf v} r \right) \left( \sqrt{Q} + 1 \right) - \mu_{\cos \theta} \left( \hat{\bf p} \cdot {\bf n} \right) \xi^{2} -\sqrt{Q} \left( \nu_r \left( \hat{\bf p} \cdot {\bf v} r \right) + \left( \mu_{\cos \theta} - \nu_{\cos \theta} \right) \left( \hat{\bf p} \cdot {\bf n} \right) \xi^{2} \right) \right] \tilde{B}^{2} }_{H_{\rm SO}\ {\rm Term\ 2a}} + \underbrace{ e^{\tilde{\mu} + \nu} \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right) \left( 2 \sqrt{Q} + 1 \right) \left[ \tilde{J} \nu_r \left( {\bf S} \cdot {\bf v} \right) - \nu_{\cos \theta} \left( {\bf S} \cdot {\bf n} \right) \xi^{2} \right] \tilde{B} }_{H_{\rm SO}\ {\rm Term\ 2b}} - \underbrace{ \tilde{J} \tilde{B}_{r} e^{\tilde{\mu} + \nu} \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right) \left( \sqrt{Q} + 1 \right) \left( {\bf S} \cdot {\bf v} \right) }_{H_{\rm SO}\ {\rm Term\ 2c}}
\end{equation}

In [13]:
%%writefile -a SEOBNR/Hamiltonian.txt

HsoTerm2 = HsoTerm2a + HsoTerm2c + HsoTerm2c

Appending to SEOBNR/Hamiltonian.txt


<a id='HsoTerm2a'></a>
### Step 4c: $H_{\rm SO}$ Term 2a \[Back to [top](#toc)\]
$$\label{HsoTerm2a}$$

We have

\begin{equation}
    H_{\rm SO}\ {\rm Term\ 2a} = \left( {\bf S} \cdot \boldsymbol{\xi} \right) \tilde{J} \left[ \mu_r \left( \hat{\bf p} \cdot {\bf v} r \right) \left( \sqrt{Q} + 1 \right) - \mu_{\cos \theta} \left( \hat{\bf p} \cdot {\bf n} \right) \xi^{2} -\sqrt{Q} \left( \nu_r \left( \hat{\bf p} \cdot {\bf v} r \right) + \left( \mu_{\cos \theta} - \nu_{\cos \theta} \right) \left( \hat{\bf p} \cdot {\bf n} \right) \xi^{2} \right) \right] \tilde{B}^{2}
\end{equation}

We define [${\bf S} \cdot \xi$](#Sdotxi), [$\tilde{J}$](#Jtilde), [$\mu_{r}$](#mur), [$\hat{\bf p} \cdot {\bf v} r$](#pdotvf), [$Q$](#Q), [$\mu_{\cos \theta}$](#mucostheta), [$\hat{\bf p} \cdot {\bf n}$](#pdotn),

<a id='Hss'></a>

# Step 5: The Spin-Spin Term $H_{\rm SS}$
$$\label{Hss}$$

From BB2010 Equation (4.19),

\begin{eqnarray}                                                                                                                                                                                            
H_{\rm SS} &=& \omega \left( {\bf S} \cdot {\bf \hat{S}}_{\rm Kerr} \right) + \frac{ e^{-3 \tilde{\mu} -\nu} \tilde{J} \omega_{r} }{ 2 \tilde{B} \left( \sqrt{Q} + 1 \right) \sqrt{Q} \xi^{2} } \left\{ -e^{\tilde{\mu} + \nu} \left( {\bf \hat{p}} \cdot {\bf v} r \right) \left( {\bf \hat{p}} \cdot {\bf \xi} r \right) \left( {\bf S} \cdot {\bf \xi} \right)
\tilde{B} + e^{2 \left( \tilde{\mu} + \nu \right)} \left( {\bf \hat{p}} \cdot {\bf \xi} r \right)^2 \left( {\bf S}
\cdot {\bf v} \right) \right. \nonumber \\
&& \left. + e^{2 \tilde{\mu}} \left( 1 + \sqrt{Q} \right) \sqrt{Q} \left( {\bf S} \cdot {\bf v} \right)\xi^2 \tilde{B}^{2} + \tilde{J} \left( {\bf \hat{p}} \cdot {\bf n} \right) \left[ \left( {\bf \hat{p}} \cdot {\bf v} r \right)
\left( {\bf S} \cdot {\bf n}\right) - \tilde{J} \left( {\bf \hat{p}} \cdot {\bf n} \right) \left( {\bf S} \cdot {\bf v} \right)\right] \xi^{2} \tilde{B}^{2} \right\} \nonumber \\
&& + \frac{ e^{-3 \tilde{\mu} - \nu} \omega_{\cos\theta} }{ 2 \tilde{B} \left( \sqrt{Q} + 1 \right) \sqrt{Q} } \left\{ -e^{2 \left( \tilde{\mu} + \nu \right)} \left( {\bf \hat{p}} \cdot {\bf \xi} r \right)^{2} \left( {\bf S} \cdot {\bf n} \right) + e^{\tilde{\mu} +\nu} \tilde{J} \left( {\bf \hat{p}} \cdot {\bf n} \right) \left( {\bf \hat{p}} \cdot {\bf \xi} r \right) \left( {\bf S} \cdot {\bf \xi} \right) \tilde{B} \right. \nonumber \\
&& \left. + \left[ \left( {\bf S} \cdot {\bf n} \right) \left( {\bf \hat{p}} \cdot {\bf v} r \right)^{2} - \tilde{J} \left( {\bf \hat{p}} \cdot {\bf n} \right) \left( {\bf S} \cdot {\bf v} \right) \left( {\bf \hat{p}} \cdot {\bf v} r\right) - e^{2 \tilde{\mu}} \left( 1 + \sqrt{Q} \right) \sqrt{Q} \left( {\bf S} \cdot {\bf n} \right) \xi^{2} \right] \tilde{B}^{2} \right\}.
\end{eqnarray}

See [this cell](#HsoHsscommon) for futher definitions.

<a id='HsoHsscommon'></a>

# Step 6: Terms Common to $H_{\rm SO}$ and $H_{\rm SS}$

From BB2010 Equations (5.48)--(5.51):

\begin{align}
\tilde{B} &= BR = \sqrt{ \Delta_{t} } \\
\tilde{B}_r &= B_r R = \frac{ \sqrt{\Delta_{r}} \Delta_{t}^{\prime} - 2 \Delta_{t} }{ 2 \sqrt{\Delta_{r} \Delta_{r}} } \\
e^{2 \tilde{\mu}} &= e^{2 \mu} R^2 = \Sigma \\
\tilde{J} &= J R = \sqrt{\Delta_{r}} \\
\end{align}

From BB2010 Equations (5.31) and (5.32):

\begin{align}
    \omega = \frac{ \tilde{\omega}_{\rm fd} }{ \Lambda_{t} } \\
    e^{2 \nu} = \frac{ \Delta_{t} \Sigma }{ \Lambda_{t} }.
\end{align}

From BB2010 Equations (5.47a)--(5.47h):

\begin{align}
    B_{r} &= \frac{ \sqrt{ \Delta_{r} } \Delta_{t}^{\prime} - 2 \Delta_{t} }{ 2 \sqrt{ \Delta_{r} \Delta_{t} } R } \\
    \omega_{r} &= \frac{ -\Lambda^{\prime}_{t} \tilde{\omega}_{\rm fd} + \Lambda_{t} \tilde{\omega}^{\prime}_{\rm fd} }{ \Lambda_{t}^{2} } \\                                                                                                                                      
    \nu_{r} &= \frac{ r }{ \Sigma } + \frac{ \varpi^{2} \left( \varpi^{2} \Delta_{t}^\prime - 4 r \Delta_{t} \right) }{ 2 \Lambda_{t} \Delta_{t} } \\
    \mu_{r} &= \frac{ r }{ \Sigma } - \frac{ 1 }{ \sqrt{ \Delta_{r} } } \\
    B_{\cos \theta} &= 0\\
    \omega_{\cos \theta} =& -\frac{2 a^2 \cos \theta \Delta_{t} \tilde{\omega}_{\rm fd} }{ \Lambda_t^{2} } \\
    \nu_{\cos \theta} &= \frac{a^2 \varpi^{2} \cos \theta \left( \varpi^{2} - \Delta{_t} \right) }{ \Lambda{_t} \Sigma } \\                                                                                                                                                                              
    \mu_{\cos \theta} &= \frac{a^2 \cos \theta }{ \Sigma }                                                                                                                                       
\end{align}

From BB2010 Equations (5.48)--(5.52):

\begin{align}
    \tilde{B} &= BR = \sqrt{ \Delta_{t} } \\
    \tilde{B}_{r} &= B_{r} R = \frac{ \sqrt{ \Delta_{r} } \Delta_{t}^{\prime} - 2 \Delta_{t} }{ 2 \sqrt{ \Delta_{r} \Delta_{t} } } \\
    e^{2 \tilde{\mu}} &= e^{2 \mu} R^{2} = \Sigma \\
    \tilde{J} &= J R = \sqrt{ \Delta_{r} } \\
    Q &= 1 + \frac{ \Delta_{r} \left( \hat{\bf p} \cdot {\bf n} \right)^{2} }{ \Sigma } + \frac{ \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right)^{2} \Sigma }{ \Lambda_{t} \sin^{2} \theta } + \frac{ \left( \hat{\bf p} \cdot {\bf v} r \right)^{2} }{ \Sigma \sin^{2} \theta }
\end{align}

<a id='HdTerms'></a>

# Step 8: The $H_{\rm D}$ Terms \[Back to [top](#toc)\]
$$\label{HdTerms}$$

Recall

\begin{equation}
    H_{\rm D} = H_{\rm D}\ {\rm coeffecient} * H_{\rm D}\ {\rm sum}.
\end{equation}

In this step we break down each of [$H_{\rm D}\ {\rm coefficient}$](#Hdcoeff) and [$H_{\rm D}\ {\rm sum}$](#Hdsum).

<a id='Hdcoeff'></a>

## Step 8a: $H_{\rm D}$ Coefficient \[Back to [top](#toc)\]
$$\label{Hdcoeff}$$

We have

\begin{equation}
    H_{\rm D}\ {\rm coefficient} = -\frac{ \eta }{ 2 r^{3} },
\end{equation}

and to save a few FLOPs we'll write

\begin{equation}
    H_{\rm D}\ {\rm coefficient} = -\frac{ \eta u^{3} }{ 2 }.
\end{equation}

We define [$\eta$](#mass_terms) and [$u$](#radius_terms) elsewhere.

In [14]:
%%writefile -a SEOBNR/Hamiltonian.txt

Hdcoeff = -sp.Rational(1,2)*eta*u3

Appending to SEOBNR/Hamiltonian.txt


<a id='Hdsum'></a>

## Step 8b: $H_{\rm D}$ Sum \[Back to [top](#toc)\]
$$\label{Hdsum}$$

We write

\begin{align}
    H_{\rm D}\ {\rm sum} &= \left( \delta^{ij} - 3 n^{i} n^{j} \right) S^{*}_{i} S^{*}_{j} \\
        &= \underbrace{\delta^{ij} S^{*}_{i} S^{*}_{j}}_{\rm Term\ 1} - \underbrace{3 n^{i} n^{j} S^{*}_{i} S^{*}_{j}}_{\rm Term\ 2}.
\end{align}

In [15]:
%%writefile -a SEOBNR/Hamiltonian.txt

Hdsum = HdsumTerm1 - HdsumTerm2

Appending to SEOBNR/Hamiltonian.txt


### $H_{\rm D}$ Sum Term 1

We have

\begin{equation}
    H_{\rm D}\ {\rm sum\ Term\ 1} = \delta^{ij} S^{*}_{i} S^{*}_{j}
\end{equation}

where $\delta^{ij}$ is the Kronecker delta:

\begin{equation}
    \delta_{ij} = \left\{ \begin{array}{cc}
        0, & i \not= j \\
        1, & i = j. \end{array} \right.
\end{equation}

Thus we have

\begin{equation}
    H_{\rm D}\ {\rm sum\ Term\ 1} = S^{*}_{1} S^{*}_{1} + S^{*}_{2} S^{*}_{2} + S^{*}_{3} S^{*}_{3}
\end{equation}

We define [${\bf S}^{*}$](#Hreal_spin_combos) as a combination of spins.

In [16]:
%%writefile -a SEOBNR/Hamiltonian.txt

HdsumTerm1 = Sstar1**2 + Sstar2**2 + Sstar3**2

Appending to SEOBNR/Hamiltonian.txt


### $H_{\rm D}$ Sum Term 2

We have

\begin{align}
    H_{\rm D}\ {\rm sum\ Term\ 2} &= 3 n^{i} n^{j} S^{*}_{i} S^{*}_{j} \\
        &= 3 \left( {\bf S}^{*} \cdot {\bf n} \right)^{2} \\
\end{align}


We define [${\bf S}^{*} \cdot {\bf n}$](#Sstardotn) elsewhere.

In [17]:
%%writefile -a SEOBNR/Hamiltonian.txt

HdsumTerm2 = 3*Sstardotn**2

Appending to SEOBNR/Hamiltonian.txt


<a id='momentum_terms'></a>

# Step N-5: The Momentum Terms \[Back to [top](#toc)\]
$$\label{momentum_terms}$$

We collect here a variety of terms dependent on the momentum ${\bf p}$.

<a id='Qminus1'></a>

## Qminus1 \[Back to [top](#toc)\]
$$\label{Qminus1}$$

First, we specify Qminus1:

\begin{equation}
    {\rm Qminus1} = Q - 1.
\end{equation}

We define [$Q$](#Q) elsewhere.

In [18]:
%%writefile -a SEOBNR/Hamiltonian.txt

Qminus1 = Q - 1

Appending to SEOBNR/Hamiltonian.txt


<a id='Q'></a>

## Q \[Back to [top](#toc)\]
$$\label{Q}$$

From BB2010 Equation (5.52),

\begin{equation}
    Q = 1 + \underbrace{ \frac{ \Delta_{r} }{ \Sigma } \left( \hat{\bf p} \cdot {\bf n} \right)^{2} }_{\rm DrSipn2} + \underbrace{ \frac{ \Sigma }{ \Lambda_t \sin^{2} \theta } }_{\rm Q\ coefficient\ 1} \left( \smash[b]{ \underbrace{ \hat{\bf p} \cdot \boldsymbol{\xi} r }_{\rm pdotxir} } \right)^{2} + \underbrace{ \frac{ 1 }{ \Sigma \sin^{2} \theta } }_{\rm Q\ coefficient\ 2} \left( \smash[b]{ \underbrace{ \hat{\bf p} \cdot {\bf v} r }_{\rm pdotvr} } \right)^{2};
\end{equation}

We group togther $\frac{ \Delta_{r} }{ \Sigma } \left( \hat{\bf p} \cdot {\bf n} \right)^{2}$ as [DrSipn2](#DrSipn2), $\frac{ \Sigma }{ \Lambda_t \sin^{2} \theta }$ as [Q coefficient 1](#Qcoeff1), $\hat{\bf p} \cdot \boldsymbol{\xi} r$ as [$pdotxir$](#pdotxir), $\frac{ 1 }{ \Sigma \sin^{2} \theta }$ as [Q coefficient 2](#Qcoeff2), and $\hat{\bf p} \cdot {\bf v} r$ as [pdotvr](#pdotvr).

In [19]:
%%writefile -a SEOBNR/Hamiltonian.txt

Q = 1 + DrSipn2 + Qcoeff1*pdotxir + Qcoeff2*pdotvr*pdotvr

Appending to SEOBNR/Hamiltonian.txt


<a id='DrSipn2'></a>

## DrSipn2 \[Back to [top](#toc)\]
$$\label{DrSipn2}$$

We'll write

\begin{equation}
    \frac{ \Delta_{r} }{ \Sigma } \left( \hat{\bf p} \cdot {\bf n} \right)^{2} = \Delta_{r} \Sigma^{-1} * \left( \hat{\bf p} \cdot {\bf n} \right)^{2}.
\end{equation}

We define $\Delta_{r}$ as [Deltar](#Deltar), $\Sigma^{-1}$ as [Sigmainv](#Sigmainv), and $\hat{\bf p} \cdot {\bf n}$ as [pdotn](#pdotn) elsewhere.

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

DrSipn2 = Deltar*Sigmainv*pdotn*pdotn

<a id='Sigmainv'></a>

## $\Sigma^{-1}$ \[Back to [top](#toc)\]
$$\label{Sigmainv}$$

We'll write

\begin{equation}
    {\rm Sigmainv = \frac{ 1 }{ \Sigma }.
\end{equation}

We define [$\Sigma$](#Sigma) elsewhere.

<a id='Deltar'></a>

## $\Delta_{r}$ \[Back to [top](#toc)\]
$$\label{Deltar}$$

From BB2010 Equation (5.38),

\begin{equation}
    \Delta_{r} = \Delta_{t} D^{-1}(u).
\end{equation}

We denote $\Delta_{t}$ as [Deltat](#Deltat) and $D^{-1}(u)$ as [Dinv](#Dinv).

In [40]:
%%writefile -a SEOBNR/Hamiltonian.txt

Deltar = Deltat*Dinv

Appending to SEOBNR/Hamiltonian.txt


<a id='Dinv'></a>

## $D^{-1}$ \[Back to [top](#toc)\]
$$\label{Dinv}$$

From BB2010 Equation (5.383),

\begin{equation}
    D^{-1} = 1 + \log \left[ 1 + 6 \eta u^{2} + 2 \left( 26 - 3 \eta \right) \eta u^{3} \right].
\end{equation}

We define [eta](#mass_terms), [u2](#radius_terms), and [u3](#radius_terms) elsewhere.

In [50]:
%%writefile -a SEOBNR/Hamiltonian.txt

Dinv = 1 + sp.log(1 + 6*eta*u2 + 2*(26 - 3*eta)*eta*u3)

Appending to SEOBNR/Hamiltonian.txt


<a id='pdotn'></a>

## $\hat{\bf p} \cdot {\bf n}$ \[Back to [top](#toc)\]
$$\label{pdotn}$$

We have

\begin{align}
    {\rm pdotn} &= \hat{\bf p} \cdot {\bf n} \\
        &= p_{1} n_{1} + p_{2} n_{2} + p_{3} n_{3}
\end{align}

Later we define [${\bf n}$](#n).

In [39]:
%%writefile -a SEOBNR/Hamiltonian.txt

pdotn = p1*n1 + p2*n2 + p3*n3

Appending to SEOBNR/Hamiltonian.txt


<a id='Deltat'></a>

## $\Delta_{t}$ \[Back to [top](#toc)\]
$$\label{Deltat}$$

From BB2010 Equation (5.71), we have

\begin{equation}
    \Delta_{t} = r^{2} \Delta_{u}.
\end{equation}

We denote $r^{2}$ by [r2](#radius_terms) and $\Delta_{u}(u)$ by [Deltau](#Deltau).

In [43]:
%%writefile -a SEOBNR/Hamiltonian.txt

Deltat = r2*Deltau

Appending to SEOBNR/Hamiltonian.txt


<a id='Deltau'></a>

## $\Delta_{u}$ \[Back to [top](#toc)\]
$$\label{Deltau}$$

From BB2010 Equation (5.73), we have

\begin{equation}
    \Delta_u = \bar{\Delta}_{u} (u) \left[ \smash[b]{\underbrace{ 1 + \eta \Delta_{0} + \eta \log \left( 1 + \Delta_{1} u + \Delta_{2} u^{2} + \Delta_{3} u^{3} + \Delta_{4} u^{4} \right) }_{\Delta_{u}\ {\rm calibration\ term}}} \vphantom{\underbrace{1}_{n}} \right]
\end{equation}

We denote $\bar{\Delta}_{u}$ by [Deltaubar](#Deltaubar) and define [$\Delta_{u}$ calibration term](#Deltaucalib) elsewhere.  <font color='red'>Note that we take the absolute value of $\Delta_{u}$ in the LALSuite code (see LALSimIMRSpinEOBHamiltonianPrec.c line 279.</font>

In [44]:
%%writefile -a SEOBNR/Hamiltonian.txt

Deltau = Deltaubar*Deltaucalib

Appending to SEOBNR/Hamiltonian.txt


<a id='Deltaucalib'></a>

## $\Delta_{u}$ Calibration Term \[Back to [top](#toc)\]
$$\label{Deltaucalib}$$

From BB2010 Equation (5.73), we have

\begin{align}
    \Delta_u\ {\rm calibration term} &= 1 + \eta \Delta_{0} + \eta \log \left( 1 + \Delta_{1} u + \Delta_{2} u^{2} + \Delta_{3} u^{3} + \Delta_{4} u^{4} \right) \\
        &= 1 + \eta \left[ \Delta_{0} + \log \left( 1 + \Delta_{1} u + \Delta_{2} u^{2} + \Delta_{3} u^{3} + \Delta_{4} u^{4} \right) \right].
\end{align}

We define [$u$](#radius_terms) elsewhere.  Each of $\Delta_{i}$, $i \in \left\{0, 1, 2, 3, 4\right\}$ are defined [here](#calib_coeffs).  <font color='red'>Note that in the LALSuite code (see LALSimIMRSpinEOBHamiltonianPrec.c line 274 there are two more terms; whence do they originate?</font>

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

Deltaucalib = 1 + eta*(Delta0 + sp.log(1 + Delta1*u + Delta2*u2 + Delta3*u3 + Delta4*u4))

<a id='calib_coeffs'></a>

## Calibration Coefficients $\Delta_{i}$, $i \in \left\{0, 1, 2, 3, 4\right\}$ \[Back to [top](#toc)\]
$$\label{calib_coeffs}$$

From BB2010 Equations (5.77)--(5.81), we have

\begin{align}
    \Delta_{0} &= K \left( \eta K - 2 \right) \\
    \Delta_{1} &= -2 \left( \eta K - 1 \right) \left( K + \Delta_{0} \right) \\
    \Delta_{2} &= \frac{1}{2} \Delta_{1} \left( -4 \eta K + \Delta_{1} + 4 \right) - \frac{ a^{2} }{ M^{2} } \left( \eta K - 1 \right)^{2} \\
    \Delta_{3} &= \frac{1}{3} \left[ -\Delta_{1}^{3} + 3 \left( \eta K - 1 \right) \Delta_{1}^{2} + 3 \Delta_{2} \Delta_{1} - 6 \left( \eta K - 1 \right) \left( -\eta K + \Delta_{2} + 1 \right) - 3 \frac{ a^{2} }{ M^{2} } \left( \eta K - 1 \right)^{2} \Delta_{1} \right] \\
    \Delta_{4} &= \frac{1}{12} \left\{ 6 \frac{ a^{2} }{ M^{2} } \left( \Delta_{1}^{2} - 2 \Delta_{2} \right) \left( \eta K - 1 \right)^{2} + 3 \Delta_{1}^{4} - 8 \left( \eta K - 1 \right) \Delta_{1}^{3} - 12 \Delta_{2} \Delta_{1}^{2} + 12 \left[ 2 \left( \eta K - 1 \right) \Delta_{2} + \Delta_{3} \right] \Delta_{1} + 12 \left( \frac{94}{3} - \frac{41}{32} \pi^{2} \right) \left( \eta K - 1 \right)^{2} + 6 \left[ \Delta_{2}^{2} - 4 \Delta_{3} \left( \eta K - 1 \right) \right] \right\}
\end{align}

We define [$K$](#K), [$\eta$](#mass_terms), [$a$](#a_terms), and [$M$](#mass_terms), [etaKminus1](#etaKminus1), and [a2M2inv](#a2M2inv) elsewhere.

In [48]:
%%writefile -a SEOBNR/Hamiltonian.txt

Delta0 = K*(eta*K - 2)
Delta1 = -2*etaKminus1*(K + Delta0)
Delta2 = sp.Rational(1,2)*Delta1*(-4*eta*K + Delta1 + 4) - a2M2inv*etaKminus1**2
Delta3 = sp.Rational(1,3)*(-Delta1***3 + 3*(etaKminus1*Delta1**2 + 3*Delta2*Delta1 - 6*etaKminus1*(-eta*K + Delta2 + 1)
                                            - 3*a2M2inv*etaKminus1**2*Delta1))
Delta4 = sp.Rational(1,12)*(6*a2M2inv*(\Delta1**2 - 2*Delta2)*etaKminus1**2 + 3*Delta1**4 - 8*etaKminus1*Delta1**3
                           -12*Delta2*Delta1**2 + 12*(2*etaKminus1*Delta2 + Delta2)*Delta1
                           + 12*(sp.Rational(94,3) - sp.Rational(41,32)*pi**2)*etaKminus1**2
                           + 6*(Delta2**2 - 4*Delta3*etaKminus1))

Appending to SEOBNR/Hamiltonian.txt


<a id='a2M2inv'></a>

## a2M2inv \[Back to [top](#toc)\]
$$\label{a2M2inv}$$

We have

\begin{equation}
    {\rm a2M2inv} = \frac{ a^{2} }{ M^{2} }
\end{equation}

We define [a2](#a_terms) and [$M$](#mass_terms) elsewhere.

In [49]:
%%writefile -a SEOBNR/Hamiltonian.txt

a2M2inv = a2/(M*M)

Appending to SEOBNR/Hamiltonian.txt


<a id='Deltaubar'></a>

## $\bar{\Delta}_{u}$ \[Back to [top](#toc)\]
$$\label{Deltaubar}$$

From BB2010 Equation (5.75), we have

\begin{equation}
    \bar{\Delta}_u = \frac{ a^{2} u^{2} }{ M^{2} } + \frac{ 1 }{ \eta K - 1 } \left( 2 u + \frac{ 1 }{ \eta K - 1 } \right)
\end{equation}

We denote $a^{2}$ by [a2](#a_terms), $u^{2}$ by [u2](#radius_terms), and $\frac{ 1 }{ \eta K - 1 }$ by [etaKminus1inv](#etaKminus1).

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

Deltaubar = s2*u2/(M*M) + etaKminus1inv*(2u + etaKminus1inv)

<a id='etaKminus1inv'></a>

## $\frac{ 1 }{ \eta K - 1 }$ \[Back to [top](#toc)\]
$$\label{etaKminus1inv}$$

We write

\begin{equation}
    {\rm etaKminus1inv} = \frac{ 1 }{ \underbrace{\eta K - 1}_{\rm etaKminus1} }
\end{equation}

We define [etaKminus1](#etaKminus1) elsewhere.

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

etaKminus1inv = 1/etaKminus1

<a id='etaKminus1'></a>

## $\eta K - 1$ \[Back to [top](#toc)\]
$$\label{etaKminus1}$$

We write

\begin{equation}
    {\rm etaKminus1} = \eta K - 1
\end{equation}

We define [$\eta$](#mass_terms) and [$K$](#K) elsewhere.

In [46]:
%%writefile -a SEOBNR/Hamiltonian.txt

etaKminus1 = eta*K - 1

Appending to SEOBNR/Hamiltonian.txt


<a id='K'></a>

## $K$ \[Back to [top](#toc)\]
$$\label{K}$$

Following the discussion in the two paragrahs after BB2010 Equation (6.11), we have

\begin{equation}
    K = 1.4467 \left( 1 - 4 \eta \right)^{2} + 4 \left( 1 - 2 \eta \right) \eta.
\end{equation}

We define [$\eta$](#mass_terms) elsewhere.

In [47]:
%%writefile -a SEOBNR/Hamiltonian.txt

K = 1.4467*(1 - 4*eta)**2 + 4*(1 - 2*eta)*eta

Appending to SEOBNR/Hamiltonian.txt


<a id='a_terms'></a>

## $a$ and $a^{2}$ \[Back to [top](#toc)\]
$$\label{a_terms}$$

From BB2010 Equation (4.9), we have

\begin{equation}
    a = \frac{ \left\lvert {\bf S}_{\rm Kerr} \right\rvert }{ M } = \frac{ \sqrt{ {\bf S}_{\rm Kerr}^{1} {\bf S}_{\rm Kerr}^{1} + {\bf S}_{\rm Kerr}^{2} {\bf S}_{\rm Kerr}^{2} + {\bf S}_{\rm Kerr}^{3} {\bf S}_{\rm Kerr}^{3} } }{ M }
\end{equation}

whence

\begin{equation}
    a^{2} = \frac{ {\bf S}_{\rm Kerr}^{1} {\bf S}_{\rm Kerr}^{1} + {\bf S}_{\rm Kerr}^{2} {\bf S}_{\rm Kerr}^{2} + {\bf S}_{\rm Kerr}^{3} {\bf S}_{\rm Kerr}^{3} }{ M^{2} }.
\end{equation}

We define [${\bf S}_{\rm Kerr}$](#Hreal_spin_combos) elsewhere.

In [45]:
%%writefile -a SEOBNR/Hamiltonian.txt

a2 = (Skerr1*Skerr1 + Skerr2*Skerr2 + Skerr3*Skerr3) / (M*M)
a = sp.sqrt(a2)

Appending to SEOBNR/Hamiltonian.txt


<a id='alphabetagamma'></a>

# Step N-4: The Metric-Dependent Terms $\alpha$, $\beta^{i}$, and $\gamma^{ij}$ \[Back to [top](#toc)\]
$$\label{alphabetagamma}$$

BB2010 specifies, in Equations (5.44)--(5.46), that for a spacetime metric $g$

\begin{align}
    \alpha &= \frac{ 1 }{ \sqrt{ - g^{tt} }} \\
    \beta^{i} &= \frac{ g^{ti} }{ g^{tt} } \\
    \gamma^{ij} &= g^{ij} - \frac{ g^{ti} g ^{tj} }{ g^{tt} }.
\end{align}

To determine [$H_{\rm real}$](#Hreal) we use the [deformed Kerr metric](#metric).

<a id='Hreal_spin_combos'></a>

# Step N-3: $H_{\rm real}$ Spin Combinations ${\bf S}^{*}$ and ${\bf S}_{\rm Kerr}$ \[Back to [top](#toc)\]
$$\label{Hreal_spin_combos}$$

From BB2010 Equation (5.63):

\begin{equation}
    {\bf S}^{*} = \boldsymbol{\sigma}^{*} + \frac{ 1 }{ c^{2} } \boldsymbol{\Delta}_{\sigma^{*}}.
\end{equation}

We define $\boldsymbol{\sigma}^{*}$ [here](#Heff_spin_combos) and $\boldsymbol{\Delta}_{\sigma^{*}}$ [here](#Deltasigmastar).

From BB2010 Equation (5.64):

\begin{equation}
    {\bf S}_{\rm Kerr} = \boldsymbol{\sigma} + \frac{ 1 }{ c^{2} } \boldsymbol{\Delta}_{\sigma}.
\end{equation}

We define $\boldsymbol{\sigma}$ [here](#Heff_spin_combos).  In BB2010 Equation (5.67), $\boldsymbol{\Delta}_{\sigma} = 0$.  Furthermore, we take $c = 1$.

In [21]:
%%writefile -a SEOBNR/Hamiltonian.txt

Sstar1 = sigmastar1 + Deltasigmastar1
Sstar2 = sigmastar2 + Deltasigmastar2
Sstar3 = sigmastar3 + Deltasigmastar3
Skerr1 = sigma1
Skerr2 = sigma2
Skerr3 = sigma3

Appending to SEOBNR/Hamiltonian.txt


<a id='Heff_spin_combos'></a>

# Step N-2: $H_{\rm eff}$ Spin Combinations $\boldsymbol{\sigma}$ and $\boldsymbol{\sigma}^{*}$ \[Back to [top](#toc)\]
$$\label{Heff_spin_combos}$$

From BB2010 Equation (5.2):

\begin{equation}
    \boldsymbol{\sigma} = {\bf S}_{1} + {\bf S}_{2}.
\end{equation}

From BB2010 Equation (5.3):

\begin{equation}
    \boldsymbol{\sigma}^{*} = \frac{ m_{2} }{ m_{1} } {\bf S}_{1} + \frac{ m_{1} }{ m_{2} }{\bf S}_{2}.
\end{equation}

These are both written completely in terms of intrinsic physical parameters.

In [22]:
%%writefile -a SEOBNR/Hamiltonian.txt

sigma1 = S1x + S2x
sigma2 = S1y + S2y
sigma3 = S3z + S3z
sigmastar1 = m2/m1*S1x + m1/m2*S2x
sigmastar2 = m2/m1*S1y + m1/m2*S2y
sigmastar3 = m2/m1*S1z + m1/m2*S2z

Appending to SEOBNR/Hamiltonian.txt


<a id='mass_terms'></a>

# Step N-1: Mass Terms $\mu$, $M$, and $\eta$ \[Back to [top](#toc)\]
$$\label{mass_terms}$$

Each of the terms $\mu$, $M$, and $\eta$ are combinations of the massess $m_{1}$, $m_{2}$ and are defined in the discussion before BB2010 Equation (5.1):

\begin{align}
    M &= m_{1} + m_{2} \\
    \mu &= \frac{ m_{1} m_{2} }{ M } \\
    \eta &= \frac{ \mu }{ M }.
\end{align}

Written completely in terms of the intrinsic physical parameters,

\begin{align}
    \mu &= \frac{ m_{2} m_{2} }{ m_{1} + m_{2} } \\
    \eta &= \frac{ m_{1} m_{2} }{ \left( m_{1} + m_{2} \right)^{2} }.
\end{align}

In [23]:
%%writefile -a SEOBNR/Hamiltonian.txt

eta = mu/M
mu = m1*m2/M
M = m1 + m2

Appending to SEOBNR/Hamiltonian.txt


<a id='radius_terms'></a>

# Step ?: Radius Terms $u$ \[Back to [top](#toc)\]
$$\label{radius_terms}$$

Each of the terms $u$, $u2$, $u3$, $u4$, and $u5$ are powers of $r^{-1}$:

\begin{align}
    u &= \frac{ 1 }{ r } \\
    u2 &= u^{2} \\
    u3 &= u^{3} \\
    u4 &= u^{4} \\
    u5 &= u^{5}.
\end{align}

To save a few FLOPs, we also collect powers of $r$:

\begin{align}
    r2 &= r^{2} \\
    r3 &= r^{3}.
\end{align}

In [41]:
%%writefile -a SEOBNR/Hamiltonian.txt

u = 1/r
u2 = u*u
u3 = u2*u
u4 = u3*u
u5 = u4*u
r2 = r*r
r3 = r2*r

Appending to SEOBNR/Hamiltonian.txt


<a id='metric'></a>

# Step N: The Metric $g$ of Choice \[Back to [top](#toc)\]
$$\label{metric}$$

In BB2010 Equations (5.36a)--(5.36e), we are given the appropriate choice of metric via

\begin{align}
    g^{tt} &= -\frac{ \Lambda_{t} }{ \Delta_{t} \Sigma } \\
    g^{rr} &= \frac{ \Delta_{r} }{ \Sigma } \\
    g^{\theta\theta} &= \frac{ 1 }{ \Sigma } \\
    g^{\phi\phi} &= \frac{ 1 }{ \Lambda_{t} } \left( -\frac{ \tilde{\omega}_{\rm fd}^{2} }{ \Delta_{t} \Sigma } + \frac{ \Sigma }{ \sin^{2} \theta } \right) \\
    g^{t\phi} &= -\frac{ \tilde{\omega}_{\rm fd} }{ \Delta_{t} \Sigma }
\end{align}

We define [$\Lambda_{t}$](#Lambda), [$\Delta_{t}$](#Deltat), [$\Sigma$](#Sigma), [$\Delta_{r}$](#Deltar), [$\tilde{\omega}_{\rm fd}$](#omegatilde) elsewhere.

In [25]:
%%writefile -a SEOBNR/Hamiltonian.txt

gtt = -Lambda/Deltat/Sigma
grr = Deltar/Sigma
ghh = 1/Sigma
gpp = 1/Lambda*(-omegatilda**2/Deltat/Sigma + Sigma/(sintheta**2))
gtp = -omegatilde/Deltat/Sigma

Appending to SEOBNR/Hamiltonian.txt


From Equations (5.73)--(5.76) we have

\begin{align}
\Delta_{u} &= \bar{\Delta}_u(u) \left[ 1 + \eta \Delta_0 + \eta \log \left( 1 + \Delta_1 u + \Delta_2 u^2 + \Delta_3 u^3 + \Delta_4 u^4 \right) \right] \\
    \bar{\Delta}_u(u) &= \frac{ a^{2} u^{2} }{ M^{2} } + \frac{ 2 u }{ \eta K - 1 } + \frac{ 1 }{ \left( \eta K - 1 \right)^{2} }
\end{align}

where $K$ is an adjustable parameter.  The coefficients $\Delta_{0}, \Delta_{1}, ..., \Delta_{4}$ are given in BB2010 Equations (5.77)--(5.81):

\begin{align}
    \Delta_0 &= K \left( \eta K - 2 \right) \\
    \Delta_1 &= -2 \left( \eta K - 1 \right) \left( K + \Delta_0 \right) \\
    \Delta_2 &= \frac{1}{2} \Delta_1 \left( -4 \eta K + \Delta_1 + 4 \right) - \frac{ a^{2} }{ M^{2} } \left( \eta K - 1 \right)^{2} \Delta_0 \\
    \Delta_3 &= \frac{1}{3} \left[ -\Delta_1^{3} + 3 \left( \eta K - 1 \right) \Delta_1^{2} + 3 \Delta_2 \Delta_1 - 6 \left( \eta K - 1 \right) \left( -\eta K + \Delta_2 + 1 \right) - 3 \frac{ a^{2} }{ M^{2} } \left( \eta K - 1 \right)^{2} \Delta_1 \right] \\
    \Delta_4 &= \frac{1}{12} \left\{ 6 \frac{ a^{2} }{ M^{2} } \left( \Delta_1^{2} - 2 \Delta_2 \right) \left( \eta K - 1 \right)^{2} + 3 \Delta_1^{4} - 8 \left( \eta K - 1 \right) \Delta_1^{3} - 12 \Delta_2 \Delta_1^{2} + 12 \left[ 2 \left( \eta K - 1 \right) \Delta_2 + \Delta_3 \right] \Delta_1 + 12 \left( \frac{94}{3} - \frac{41}{32} \pi^2 \right) \left( \eta K - 1 \right)^{2} + 6 \left[ \Delta_2^{2} - 4 \Delta_3 \left( \eta K - 1 \right) \right] \right\}
\end{align}


(5.37)--(5.40) and (5.71)--(5.72),

\begin{align}
    \Delta_{t} &= r^{2} \Delta_{u}(u) \\
    \Delta_{u}(u) &= A(u) + \frac{ a^{2} }{ M^{2} } u^2 \\
    \Delta_{r} &= \Delta_{t} D^{-1}(u) \\
    \Lambda_{t} &= \varpi^{4} - a^{2} \Delta_{t} \sin^{2} \theta \\
    \tilde{\omega}_{\rm fd} &= 2 a M r + \omega_1^{\rm fd} \eta \frac{ a M^{3} }{ r } + \omega_{2}^{\rm fd} \eta \frac{ M a^{3} }{ r }
\end{align}

with $u = \frac{ M }{ r }$ and frame-dragging strength adjustable parameters $\omega_1^{\rm fd}$, $\omega_2^{\rm fd}$.  Finally, from BB2010 Equations (5.41) and (5.42)

\begin{align}
    A(u) &= 1 - 2 u + 2 \eta u^{3} + \eta \left( \frac{94}{3} - \frac{41}{32} \pi^{2} \right) u^{4} \\
    D^{-1}(u) &= 1 + 6 \eta u^{2} + 2 \left( 26 - 3 \eta \right) \eta u^{3}.
\end{align}

<a id='Sstardotn'></a>

### ${\bf S}^{*} \cdot {\bf n}$ \[Back to [top](#toc)\]
$$\label{Sstardotn}$$

We have

\begin{equation}
    {\bf S}^{*} \cdot {\bf n} = {\bf S}^{*}_{1} n_{1} + {\bf S}^{*}_{2} n_{2} + {\bf S}^{*}_{3} n_{3}
\end{equation}

with [${\bf S}^{*}$](#Hreal_spin_combos) and [${\bf n}$](#n) defined later.

In [26]:
%%writefile -a SEOBNR/Hamiltonian.txt

Sstardotn = Sstar1*n1 + Sstar2*n2 + Sstar3*n3

Appending to SEOBNR/Hamiltonian.txt


<a id='n'></a>

## ${\bf n}$ \[Back to [top](#toc)\]
$$\label{n}$$

We have

\begin{equation}
    {\bf n} = \frac{\bf x }{ r }
\end{equation}

where $r = \left\lvert {\bf x} \right\rvert$ and ${\bf x} = (x, y, z)$.

In [27]:
%%writefile -a SEOBNR/Hamiltonian.txt

n1 = x/r
n2 = y/r
n3 = z/r

Appending to SEOBNR/Hamiltonian.txt


<a id='Deltasigmastar'></a>

### $\boldsymbol{\Delta}_{\sigma^{*}}$ \[Back to [top](#toc)\]
$$\label{Deltasigmastar}$$

We can write $\boldsymbol{\Delta}_{\sigma^{*}}$ as

\begin{equation}
    \boldsymbol{\Delta}_{\sigma^{*}} = \boldsymbol{\sigma}^{*} \left( \boldsymbol{\sigma}^{*}\ {\rm coefficient} \right) + \boldsymbol{\sigma} \left( \boldsymbol{\sigma}\ {\rm coefficient} \right)
\end{equation}

For further dissection, see [$\boldsymbol{\sigma}^{*}$](#Heff_spin_combos), [$\boldsymbol{\sigma}^{*}\ {\rm coefficient}$](#sigmastarcoeff) and [$\boldsymbol{\sigma}$](#Heff_spin_combos), and [$\boldsymbol{\sigma}\ {\rm coefficient}$](#sigmacoeff).

In [28]:
%%writefile -a SEOBNR/Hamiltonian.txt

Deltasigmastar1 = sigmastar1*sigmastarcoeff + sigma1*sigmacoeff
Deltasigmastar2 = sigmastar2*sigmastarcoeff + sigma2*sigmacoeff
Deltasigmastar3 = sigmastar3*sigmastarcoeff + sigma3*sigmacoeff

Appending to SEOBNR/Hamiltonian.txt


<a id='sigmastarcoeff'></a>

### $\boldsymbol{\sigma}^{*}$ coefficient \[Back to [top](#toc)\]
$$\label{sigmastarcoeff}$$

We will break down both $\boldsymbol{\sigma}^{*}\ {\rm coefficient}$ into three terms:

\begin{equation}
    \boldsymbol{\sigma}^{*}\ {\rm coefficient} = \boldsymbol{\sigma}^{*}\ {\rm coefficient\ Term\ 1} + \boldsymbol{\sigma}^{*}\ {\rm coefficient\ Term\ 2} + \boldsymbol{\sigma}^{*}\ {\rm coefficient\ Term\ 3}
\end{equation}

See [$\boldsymbol{\sigma}^{*}$ coefficient Term 1](#sigmastarcoeffTerm1), [$\boldsymbol{\sigma}^{*}$ coefficient Term 2](#sigmastarcoeffTerm2), and [$\boldsymbol{\sigma}^{*}$ coefficient Term 3](#sigmastarcoeffTerm3).

In [29]:
%%writefile -a SEOBNR/Hamiltonian.txt

sigmastarcoeff = sigmastarcoeffTerm1 + sigmastarcoeffTerm2 + sigmastarcoeffTerm3

Appending to SEOBNR/Hamiltonian.txt


<a id='sigmacoeff'></a>

### $\boldsymbol{\sigma}$ coefficient \[Back to [top](#toc)\]
$$\label{sigmacoeff}$$

We will break down $\boldsymbol{\sigma}\ {\rm coefficient}$ into three terms:

\begin{equation}
    \boldsymbol{\sigma}\ {\rm coefficient} = \boldsymbol{\sigma}\ {\rm coefficient\ Term\ 1} + \boldsymbol{\sigma}\ {\rm coefficient\ Term\ 2} + \boldsymbol{\sigma}\ {\rm coefficient\ Term\ 3}
\end{equation}

See [$\boldsymbol{\sigma}$ coefficient Term 1](#sigmacoeffTerm1), [$\boldsymbol{\sigma}$ coefficient Term 2](#sigmacoeffTerm2), and [$\boldsymbol{\sigma}$ coefficient Term 3](#sigmacoeffTerm3).

In [30]:
%%writefile -a SEOBNR/Hamiltonian.txt

sigmacoeff = sigmacoeffTerm1 + sigmacoeffTerm2 + sigmacoeffTerm3

Appending to SEOBNR/Hamiltonian.txt


<a id='sigmastarcoeffTerm1'></a>

### $\boldsymbol{\sigma}^{*}$ Coefficient Term 1 \[Back to [top](#toc)\]
$$\label{sigmastarcoeffTerm1}$$

We build this term from BB2011 Equation (51) with $b_{0} = 0$ (<font color='red'>why?</font>), where what is listed below is the coefficient on $\boldsymbol{\sigma}^{*}$:

\begin{align}
    \boldsymbol{\sigma}^{*}\ {\rm coefficient\ Term\ 1} &= \frac{7}{6} \eta \frac{M}{r} + \frac{1}{3} \eta \left( Q - 1 \right) - \frac{5}{2} \eta \frac{ \Delta_r }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \\
        &= \frac{ \eta }{ 12 } \left( 14 \frac{ M }{ r } + 4 \left( Q - 1 \right) - 30 \frac{ \Delta_r }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \right)
\end{align}

We group together the following: $Q-1$ as [Qminus1](#Qminus1) and $\frac{ \Delta_r }{ \Sigma }$ as [$DrSinp2$](#DrSinp2).  We define [$\eta$](#mass_terms) and [$M$](#mass_terms) elsewhere.

In [31]:
%%writefile -a SEOBNR/Hamiltonian.txt

sigmastarcoeffTerm1 = eta/12*(14*M/r + 4*Qminus1 - 30*DrSinp2)

Appending to SEOBNR/Hamiltonian.txt


<a id='sigmastarcoeffTerm2'></a>

### $\boldsymbol{\sigma}^{*}$ Coefficient Term 2 \[Back to [top](#toc)\]
$$\label{sigmastarcoeffTerm2}$$

We build this term from BB2011 Equation (52) with all $b_{i} = 0$, $i \in \left\{0, 1, 2, 3\right\}$ (<font color='red'>why?</font>), and just the coefficient on $\boldsymbol{\sigma}^{*}$.  In the LALSuite code this is the variable 'sMultiplier1':

\begin{align}
    \boldsymbol{\sigma}^{*}\ {\rm coefficient\ Term\ 2} &= \frac{1}{36} \left( 353 \eta - 27 \eta^2 \right) \left( \frac{M}{r} \right)^{2} + \frac{5}{3} \left( 3 \eta^2 \right) \frac{ \Delta_{r}^{2} }{ \Sigma^{2} } \left( {\bf n} \cdot \hat{\bf p} \right)^{4} \\
        &+ \frac{1}{72} \left( -23 \eta -3 \eta^{2} \right) \left( Q - 1 \right)^{2} + \frac{1}{36} \left( -103 \eta + 60 \eta^{2} \right) \frac{M}{r} \left( Q - 1 \right) \\
        &+ \frac{1}{12} \left( 16 \eta - 21 \eta^{2} \right) \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \left( Q - 1 \right) + \frac{1}{12} \left( 47 \eta - 54 \eta^{2} \right) \frac{M}{r} \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \\
        &= \frac{ \eta }{ 72 r^{2} } \left[ \left( 706 - 54 \eta \right) M^{2} + 360 \eta r^{2} \frac{ \Delta_{r}^{2} }{ \Sigma^{2} } \left( {\bf n} \cdot \hat{\bf p} \right)^{4} + r^{2} \left( -23 - 3 \eta \right) \left( Q - 1 \right)^{2} + \left( -206 + 120 \eta \right) M r \left( Q - 1 \right) \right. \\
        &+ \left. \left( 96 - 126 \eta \right) r^{2} \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \left( Q - 1 \right) + \left( 282 - 324 \eta \right) M r \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \right] \\
        &= \frac{ \eta }{ 72 r^{2} } \left[ 706 + r \left( -206 M \left( Q - 1 \right) + 282 M \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} + r \left( Q -1 \right) \left( 96 \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} - 23 \left( Q - 1 \right) \right) \right) \right. \\
        &+ \left. \eta \left( 54 M^{2} + r \left( 120 M \left( Q -1 \right) - 324 M \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} + r \left( 360 \frac{ \Delta_{r}^{2} }{ \Sigma^{2} } \left( {\bf n} \cdot \hat{\bf p} \right)^{4} + \left( Q - 1 \right) \left( -126 \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} - 3 \left( Q - 1 \right) \right) \right)\right) \right) \right]
\end{align}

We define [$\eta$](#mass_terms), [$M$](#mass_terms), [$Q - 1$](#Qminus1), and [$\frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2}$](#DrSinp2) elsewhere.

In [32]:
%%writefile -a SEOBNR/Hamiltonian.txt

sigmastarcoeffTerm2 = eta/(72*r**2) ( 706 + r*( -206*M*Qminus1 + 282*M*DrSinp2 + r*Qminus1*(96*DrSinp2 - 23*Qminus1))
                                   + eta*(54*M**2 + r*(120*M*Qminus1-324*M*DrSinp2 + r*(360*DrSinp2**2 + Qminus1*(-126*DrSinp2 - 3*Qminus1)))))

Appending to SEOBNR/Hamiltonian.txt


<a id='sigmastarcoeffTerm3'></a>

### $\boldsymbol{\sigma}^{*}$ Coefficient Term 3 \[Back to [top](#toc)\]
$$\label{sigmastarcoeffTerm3}$$

<font color='red'>I have no idea whence this term originates; there's not any helpful reference in the code.  Unless I'm missing something...</font>

In [33]:
%%writefile -a SEOBNR/Hamiltonian.txt

sigmastarcoeffTerm3 =

Appending to SEOBNR/Hamiltonian.txt


<a id='sigmacoeffTerm1'></a>

### $\boldsymbol{\sigma}$ Coefficient Term 1 \[Back to [top](#toc)\]
$$\label{sigmacoeffTerm1}$$

We build this term from BB2011 Equation (51) with $a_{0} = 0$ (<font color='red'>why?</font>), where what is listed below is the coefficient on $\boldsymbol{\sigma}$:

\begin{align}
    \boldsymbol{\sigma}\ {\rm coefficient\ Term\ 1} &= -\frac{2}{3} \eta \frac{ M }{ r } + \frac{1}{4} \eta \left( Q - 1 \right) - 3 \eta \frac{ \Delta_r }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \\
        &= \frac{ \eta }{ 12 } \left( -8 \frac{ M }{ r } + 3 \left( Q - 1 \right) - 36 \frac{ \Delta_r }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \right)
\end{align}

where, from the discussion after BB2011 Equation (17), $a_{0}$ and $b_{0}$ are arbitrary gauge functions of [$\eta$](#mass_terms).  <font color='red'>List other terms here once you break this up.</font>

In [34]:
%%writefile -a SEOBNR/Hamiltonian.txt

sigmacoeffTerm1 = eta/12*(-8*M/r + 3*Qminus1 - 36*DrSinp2)

Appending to SEOBNR/Hamiltonian.txt


<a id='sigmacoeffTerm2'></a>

### $\boldsymbol{\sigma}$ Coefficient Term 2 \[Back to [top](#toc)\]
$$\label{sigmacoeffTerm2}$$

We build this term from BB2011 Equation (52) with all $a_{i} = 0$, $i \in \left\{0, 1, 2, 3\right\}$ (<font color='red'>why?</font>), and just the coefficient on $\boldsymbol{\sigma}$:

\begin{align}
    \boldsymbol{\sigma}\ {\rm coefficient\ Term\ 2} &= \frac{1}{9} \left( -56 \eta -21 \eta^{2} \right) \left( \frac{ M }{ r } \right)^{2} + \frac{5}{24} \left( 27 \eta^{2} \right) \frac{ \Delta_r^{2} }{ \Sigma^{2} } \left( {\bf n} \cdot \hat{\bf p} \right)^{4} \\
        &+ \frac{1}{144} \left(-45 \eta \right) \left( Q - 1 \right)^{2} + \frac{1}{36} \left( -109 \eta + 51 \eta^{2} \right) \frac{ M }{ r } \left( Q - 1 \right) \\
        &+ \frac{1}{24} \left( 6 \eta - 39\eta^{2} \right) \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \left( Q - 1 \right) + \frac{1}{24} \left( -16 \eta - 147 \eta^{2} \right) \frac{ M }{ r } \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \\
        &= \frac{ \eta }{ 144 r^{2} } \left[ -896 M^{2} + r \left( -436 M \left( Q - 1 \right) - 96 M \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} + r \left( -45 \left( Q - 1 \right)^{2} + 36 \left( Q - 1 \right) \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \right) \right) \right. \\
        &+ \left. \eta \left( -336 M^{2} + r \left( 204 M \left( Q -1 \right) - 882 M \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} + r \left( 810 \frac{ \Delta_{r}^{2} }{ \Sigma^{2} } \left( {\bf n} \cdot \hat{\bf p} \right)^{4}  - 234 \left( Q - 1 \right) \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \right) \right) \right) \right]
\end{align}

We define [$\eta$](#mass_terms), [$M$](#mass_terms), [$Q - 1$](#Qminus1), and [$\frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2}$](#DrSinp2) elsewhere.

In [35]:
%%writefile -a SEOBNR/Hamiltonian.txt

sigmacoeffTerm2 = eta/(144*r**2)*(-896*M**2 + r*(-436*Qminus1 - 96*M*DrSinp2 + r*(-45*Qminus1**2 + 36*Qminus1*DrSinp2))
                                 + eta*(-336*M**2 + r*(204*M*Qminus1 - 882*M*DrSinp2 + r*(810*DrSinp2**2 - 234*Qminus1*DrSinp2))))

Appending to SEOBNR/Hamiltonian.txt


<a id='sigmacoeffTerm3'></a>

### $\boldsymbol{\sigma}$ Coefficient Term 3 \[Back to [top](#toc)\]
$$\label{sigmacoeffTerm3}$$

<font color='red'>I have no idea whence this term originates; there's not any helpful reference in the code.  Unless I'm missing something...</font>

In [36]:
%%writefile -a SEOBNR/Hamiltonian.txt

sigmacoeffTerm3 =

Appending to SEOBNR/Hamiltonian.txt


<a id='betapsum'></a>

### $\beta^{i} p_{i}$ \[Back to [top](#toc)\]
$$\label{betapsum}$$

We have

\begin{equation}
    \beta^{i} p_{i} = \beta^{1} p_{1} + \beta^{2} p_{2} + \beta^{3} p_{3}
\end{equation}

We define [$\beta^{1}$](#alphabetagamma), [$\beta^{2}$](#alphabetagamma), [$\beta^{3}$](#alphabetagamma) elsewhere.

In [37]:
%%writefile -a SEOBNR/Hamiltonian.txt

betapsum = beta1*p1 + beta2*p2 + beta3*p3

Appending to SEOBNR/Hamiltonian.txt


The quantity [$Q$](#momentum_terms) is a momentum term.

from BB2010 Equation (5.38)

\begin{equation}
    \Delta_{r} = \Delta_{t} D^{-1}(u);
\end{equation}

from BB2010 Equation (4.5)

\begin{equation}
    \Sigma = r^{2} + a^{2} \cos^{2} \theta;
\end{equation}

from BB2010 Equation (3.23)

\begin{equation}
    \boldsymbol{\xi} = {\bf e}_{Z} \times {\bf N},
\end{equation}

where ${\bf e}_{Z}$ is a Cartesian unit vector in the $z$-direction and ${\bf N} = {\bf n}$ from BB2010 Equation (3.37); from BB2010 Equation (5.37)

\begin{equation}
    \Delta_{t} = r^{2} \left[ A(u) + \frac{ a^{2} }{ M^{2} } u^{2} \right];
\end{equation}

from BB2010 Equation (3.39)

\begin{equation}
    {\bf v} = {\bf n} \times \boldsymbol{\xi};
\end{equation}

from BB2010 Equation (5.42)

\begin{equation}
    D^{-1}(u) = 1 + 6 \eta u^{2} + 2 \left( 26 - 3 \eta \right) \eta u^{3};
\end{equation}

from BB2010 discussion following Equation (5.40)

\begin{equation}
    u = \frac{ M }{ r };
\end{equation}

from BB2010 Equation (4.9)

\begin{equation}
    a = \frac{ \left\lvert {\bf S}_{\rm Kerr} \right\rvert }{ M };
\end{equation}

and from BB2010 Equation (5.41)

\begin{equation}
    A(u) = 1 - 2u + 2 \eta u^{3} + \eta \left( \frac{94}{3} - \frac{41}{32} \pi^{2} \right) u^{4}.
\end{equation}

Oof.

<a id='notes'></a>
# Notes \[Back to [top](#toc)\]
$$\label{notes}$$

1. First, just break down the Hamiltonian as in the paper.  Break up into blocks as appropriate.
2. Use python markdown to write string of input variables:

```python
# Step 0a: declare instrinsic system paramters (masses and spin components)
m1, m2, s1x, s1y, s1z, s2x, s2y, s2z = sp.symbols("m1 m2 s1x s1y s1z s2x s2y s2z",real=True)

```
3. Write terms to files in backwards order!  As in:

    ```python
    #First write is this:
    #%%writefile SEOBNR/Hamiltonian.txt

    ```
   and afterward

    ```python
    #%%writefile -a SEOBNR/Hamiltonian.txt

    ```
   The goal is to split up the terms so that each $\LaTeX$-expression at the bottom of a markdown cell can be easily compared to the "bigstring" expression that is SymPy compatible in a code cell directly below the markdown cell.
4. ${\cal Q}_{4}$ is quartic in $p_{i}$ and is given by Equation (4.34) of [Damour, Jaranowski, and Schaefer](https://arxiv.org/abs/gr-qc/0005034) <font color='red'> (double check this!)</font>

<a id='latex_pdf_output'></a>

# Output: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

In [38]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-SEOBNR_Documentation.ipynb
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Documentation.ipynb
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Documentation.ipynb
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Documentation.ipynb
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-SEOBNR_Documentation.ipynb to latex
[NbConvertApp] Writing 85574 bytes to Tutorial-SEOBNR_Documentation.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
